In [1]:
import os
import numpy as np
import fiona
import rasterio
from rasterio.mask import mask

In [2]:
rasterio.__version__

'1.1.5'

In [3]:
ls

geo_cut_raster_with_shp_v01.ipynb   GISOh.py
geo_cut_raster_with_shp_v02.ipynb   masked_5.tif
geo_cut_raster_with_shp_v031.ipynb  reprojetion.ipynb
GEO_cut_raster_with_shp_v04.ipynb


In [4]:
#path_data = '/mnt/sda1/gis/test/mask_wo_179_04.shp'
path_data = '/mnt/sda1/gis/test/mask_ww.shp'
#path_data_tif = '/mnt/sda1/gis/orthophotos/2018250_bischofshofen_lb_rgbi.img'
path_data_tif = '/mnt/sda1/gis/orthophotos/2018_wien/2018460_wien_lb_rgbi.img'
#path_data_tif = '/mnt/sda1/gis/Oberhoehen/ndom_oa.tif'

In [5]:
# load shapefile with polygons

#path_data = os.path.join(os.getcwd(), 'shape', 'new_2_crs.shp')
with fiona.open(path_data) as shapefile:
    geoms = [feature for feature in shapefile]

In [6]:
# load shapefile with polygons

#path_data = os.path.join(os.getcwd(), 'shape', 'new_crs.shp')
#with fiona.open(path_data) as shapefile:
#    geomss = [feature["geometry"] for feature in shapefile]

In [7]:
# load shapefile with polygons
#path_data = os.path.join(os.getcwd(), 'shape', 'new_crs.shp')
#fc = fiona.open(path_data)
#list_poly = []
#for i in range(1000):
#    list_poly.append(fc.next())

In [8]:
# load raster with information

#path_data_tif = os.path.join(os.getcwd(), 'tiff', '171_Wienerwald_ndsm.tif')
dataset = rasterio.open(path_data_tif)

In [9]:
fc.crs

NameError: name 'fc' is not defined

In [10]:
dataset.crs

CRS.from_wkt('PROJCS["Lambert Conformal Conic",GEOGCS["MGI AT (EPSG:1618)",DATUM["MGI AT (EPSG:1618)",SPHEROID["Bessel",6377397.155,299.15281310608],TOWGS84[577.326,90.129,463.919,5.137,1.474,5.297,2.4232]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitude_of_origin",47.5],PARAMETER["central_meridian",13.3333333333333],PARAMETER["standard_parallel_1",49],PARAMETER["standard_parallel_2",46],PARAMETER["false_easting",400000],PARAMETER["false_northing",400000],UNIT["meters",1],AXIS["Easting",EAST],AXIS["Northing",NORTH]]')

In [18]:
dataset.crs

CRS.from_wkt('PROJCS["Lambert Conformal Conic",GEOGCS["MGI AT (EPSG:1618)",DATUM["MGI AT (EPSG:1618)",SPHEROID["Bessel",6377397.155,299.15281310608],TOWGS84[577.326,90.129,463.919,5.137,1.474,5.297,2.4232]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitude_of_origin",47.5],PARAMETER["central_meridian",13.3333333333333],PARAMETER["standard_parallel_1",49],PARAMETER["standard_parallel_2",46],PARAMETER["false_easting",400000],PARAMETER["false_northing",400000],UNIT["meters",1],AXIS["Easting",EAST],AXIS["Northing",NORTH]]')

In [11]:
geoms[5]

{'type': 'Feature',
 'id': '5',
 'properties': OrderedDict([('fid', 6.0),
              ('OBJECTID', 4158.0),
              ('FB_KURZ', 171),
              ('FB_LANG', 'Wienerwald'),
              ('FR_KURZ', 1),
              ('FR_LANG', 'Kierling'),
              ('ABTEILUNG', 653),
              ('SHAPE_Leng', 3310.9009050525456),
              ('SHAPE_Area', 499758.7836926781)]),
 'geometry': {'type': 'Polygon',
  'coordinates': [[(611808.0200000033, 433328.4600000009),
    (611829.0200000033, 433338.2100000009),
    (611840.2199999988, 433345.44999999925),
    (611841.6000000015, 433346.33999999985),
    (611841.7899999991, 433346.4600000009),
    (611850.3200000003, 433351.44000000134),
    (611857.4799999967, 433358.6000000015),
    (611857.7199999988, 433358.8200000003),
    (611857.9799999967, 433359.0300000012),
    (611858.2400000021, 433359.2199999988),
    (611858.5200000033, 433359.3900000006),
    (611858.7800000012, 433359.5300000012),
    (611881.1400000006, 433370.710

In [12]:
geom = geoms[5]['geometry']

In [14]:
# cutting the raster
#path_data_tif = os.path.join(os.getcwd(), 'tiff', '171_Wienerwald_ndsm.tif')
with rasterio.open(path_data_tif) as src:
    out_image, out_transform = mask(src.read(1), [geom], crop=True)
out_meta = src.meta.copy()

MemoryError: Unable to allocate 79.9 GiB for an array with shape (1, 224251, 382727) and data type uint8

In [14]:
out_image, out_transform = mask(dataset, [geom], crop=True)
out_meta = dataset.meta.copy()

In [15]:
# save the resulting raster  
out_meta.update({"driver": "GTiff",
    "height": out_image.shape[1],
    "width": out_image.shape[2],
"transform": out_transform})

with rasterio.open("masked_5.tif", "w", **out_meta) as dest:
    dest.write(out_image)

In [63]:
out_image.shape

(1, 324, 167)

In [30]:
print(out_image.max())
print(out_image.min())
print(out_image.mean())

23.255981
-3.402823e+38
-inf


In [65]:
# set -3.402823e+38 to nan

out_image[out_image<-3.3e+38]=np.nan

In [66]:
# calc OH

print(np.nanmax(out_image))
print(np.nanmin(out_image))
print(np.nanmedian(out_image))
print(np.nanmean(out_image))
print(np.nanstd(out_image))

45.0
0.39501953
26.188965
25.620052
4.4658017


In [67]:
flat = out_image.flatten()

In [70]:
flat = flat[~np.isnan(flat)]

In [76]:
np.percentile(flat, 80)

28.799414062500002

In [83]:
for i in range(10, 105, 5):
    print(i , np.percentile(flat, i))

10 20.486376953125003
15 22.1305908203125
20 23.13402099609375
25 23.833984375
30 24.42498779296875
35 24.94898681640625
40 25.393023681640624
45 25.79979248046875
50 26.18896484375
55 26.55421142578125
60 26.927978515625
65 27.3056396484375
70 27.72178955078125
75 28.22198486328125
80 28.799414062500002
85 29.44000244140625
90 30.235217285156253
95 31.304010009765626
100 45.0


In [25]:
dataset.width

25373

In [26]:
dataset.height

15148

In [10]:
dataset.bounds

BoundingBox(left=589400.0086999976, bottom=464101.20250000095, right=614773.0086999976, top=479249.20250000095)

In [13]:
fc.bounds

(596140.5399999991, 408533.6499999985, 605921.3200000003, 415821.9699999988)

In [ ]:
dataset.transform

In [11]:
dataset.crs

CRS({'proj': 'lcc', 'lat_1': 46, 'lat_2': 49, 'lat_0': 47.5, 'lon_0': 13.33333333333333, 'x_0': 400000, 'y_0': 400000, 'datum': 'hermannskogel', 'units': 'm', 'no_defs': True})

In [12]:
fc.crs

{'proj': 'lcc',
 'lat_1': 46,
 'lat_2': 49,
 'lat_0': 47.5,
 'lon_0': 13.333333333,
 'x_0': 400000,
 'y_0': 400000,
 'ellps': 'bessel',
 'units': 'm',
 'no_defs': True}

In [ ]:
fc.bounds

In [ ]:
dataset.bounds

In [ ]:
"""Coordinate and geometry warping and reprojection"""

from fiona._transform import _transform, _transform_geom


def transform(src_crs, dst_crs, xs, ys):
    """Transform coordinates from one reference system to another.
    Parameters
    ----------
    src_crs: str or dict
        A string like 'EPSG:4326' or a dict of proj4 parameters like
        {'proj': 'lcc', 'lat_0': 18.0, 'lat_1': 18.0, 'lon_0': -77.0}
        representing the coordinate reference system on the "source"
        or "from" side of the transformation.
    dst_crs: str or dict
        A string or dict representing the coordinate reference system
        on the "destination" or "to" side of the transformation.
    xs: sequence of float
        A list or tuple of x coordinate values. Must have the same
        length as the ``ys`` parameter.
    ys: sequence of float
        A list or tuple of y coordinate values. Must have the same
        length as the ``xs`` parameter.
    Returns
    -------
    xp, yp: list of float
        A pair of transformed coordinate sequences. The elements of
        ``xp`` and ``yp`` correspond exactly to the elements of the
        ``xs`` and ``ys`` input parameters.
    Examples
    --------
    >>> transform('EPSG:4326', 'EPSG:26953', [-105.0], [40.0])
    ([957097.0952383667], [378940.8419189212])
    """
    # Function is implemented in the _transform C extension module.
    return _transform(src_crs, dst_crs, xs, ys)


def transform_geom(
        src_crs, dst_crs, geom,
        antimeridian_cutting=False, antimeridian_offset=10.0, precision=-1):
    """Transform a geometry obj from one reference system to another.
    Parameters
    ----------
    src_crs: str or dict
        A string like 'EPSG:4326' or a dict of proj4 parameters like
        {'proj': 'lcc', 'lat_0': 18.0, 'lat_1': 18.0, 'lon_0': -77.0}
        representing the coordinate reference system on the "source"
        or "from" side of the transformation.
    dst_crs: str or dict
        A string or dict representing the coordinate reference system
        on the "destination" or "to" side of the transformation.
    geom: obj
        A GeoJSON-like geometry object with 'type' and 'coordinates'
        members.
    antimeridian_cutting: bool, optional
        ``True`` to cut output geometries in two at the antimeridian,
        the default is ``False`.
    antimeridian_offset: float, optional
        A distance in decimal degrees from the antimeridian, outside of
        which geometries will not be cut.
    precision: int, optional
        Optional rounding precision of output coordinates, in number
        of decimal places.
    Returns
    -------
    obj
        A new GeoJSON-like geometry with transformed coordinates. Note
        that if the output is at the antimeridian, it may be cut and 
        of a different geometry ``type`` than the input, e.g., a
        polygon input may result in multi-polygon output.
    Examples
    --------
    >>> transform_geom(
    ...     'EPSG:4326', 'EPSG:26953',
    ...     {'type': 'Point', 'coordinates': [-105.0, 40.0]})
    {'type': 'Point', 'coordinates': (957097.0952383667, 378940.8419189212)}
    """
    # Function is implemented in the _transform C extension module.
    return _transform_geom(
        src_crs, dst_crs, geom,
        antimeridian_cutting, antimeridian_offset, precision)

In [ ]:
crs_old = {'proj': 'lcc',
 'lat_1': 46,
 'lat_2': 49,
 'lat_0': 48,
 'lon_0': 13.333333333,
 'x_0': 400000,
 'y_0': 400000,
 'ellps': 'bessel',
 'units': 'm',
 'no_defs': True}

crs_new = {'proj': 'lcc',
 'lat_1': 46,
 'lat_2': 49,
 'lat_0': 47.5,
 'lon_0': 13.333333333,
 'x_0': 400000,
 'y_0': 400000,
 'ellps': 'bessel',
 'units': 'm',
 'no_defs': True}

In [ ]:
transform_geom(crs_old, crs_new, fc)